<a href="https://colab.research.google.com/github/shahiryar/light-recommendation-system/blob/main/tf-MovieLens_RecSystem_2TRM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommendation System from Movie Lens: Retrieval

In [ ]:
! pip install tensorflow-recommenders

In [ ]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

- Read Data

In [ ]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/0.1.1.incompleteSIWY9G/movielens-train.tfrecord*...:…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


- Build userid and movvie titles vocabulary

In [ ]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

- Defining Model

In [ ]:
class MovieLensModel(tfrs.Model):
  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()
    #use and movie representation initialization
    self.user_model = user_model #Querry Tower
    self.movie_model = movie_model #Candidate Tower

    #initialize Retrieval task
    self.task = task #dot product of the Querry and Candidate tower

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    user_embeddings = self.user_model(features['user_id'])
    movie_embeddings = self.movie_model(features['movie_title'])

    return self.task(user_embeddings, movie_embeddings)


- define two models and the Retrieval task

In [ ]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])

movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)
])

#defining Objective

task = tfrs.tasks.Retrieval(
    metrics= tfrs.metrics.FactorizedTopK(
        movies.batch(128).map(movie_model)
    )
)

- Create Model

In [ ]:
model = MovieLensModel(user_model, movie_model, task)
model.compile(
    optimizer=tf.keras.optimizers.Adagrad(0.5)
)

- train model

In [ ]:
model.fit(ratings.batch(4096), epochs=3)

Epoch 1/3
25/25 [==============================] - 38s 1s/step - factorized_top_k/top_1_categorical_accuracy: 8.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0016 - factorized_top_k/top_10_categorical_accuracy: 0.0048 - factorized_top_k/top_50_categorical_accuracy: 0.0458 - factorized_top_k/top_100_categorical_accuracy: 0.1034 - loss: 33080.0432 - regularization_loss: 0.0000e+00 - total_loss: 33080.0432
Epoch 2/3
25/25 [==============================] - 33s 1s/step - factorized_top_k/top_1_categorical_accuracy: 2.2000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0052 - factorized_top_k/top_10_categorical_accuracy: 0.0148 - factorized_top_k/top_50_categorical_accuracy: 0.1060 - factorized_top_k/top_100_categorical_accuracy: 0.2119 - loss: 31009.7812 - regularization_loss: 0.0000e+00 - total_loss: 31009.7812
Epoch 3/3
25/25 [==============================] - 34s 1s/step - factorized_top_k/top_1_categorical_accuracy: 5.6000e-04 - factorized_top_k/top_5_categorical_accu

In [14]:
# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Top 3 recommendations for user 42: [b'Rent-a-Kid (1995)' b'House Arrest (1996)' b'Just Cause (1995)']


In [15]:
path = '/content/drive/MyDrive/ML Models'
index.save(path)